In [188]:
# auto reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [189]:
import pandas as pd
from mtt.simulator import position_image
from mtt.utils import compute_ospa
from tqdm import tqdm

In [190]:
filter_name = "lmb"
cols = ["scale", "simulation_idx", "step_idx", "posx_m", "posy_m", "exist_prob"]

df_truth = pd.read_csv(f"../data/sim_data/{filter_name}_truth.csv")
df_filter = pd.read_csv(f"../data/sim_data/{filter_name}_estimates.csv")
df_combined = (
    pd.concat([df_truth, df_filter], axis=0, keys=["truth", "filter"])
    .reset_index(level=0, names=["truth"])
    .rename(columns={"time": "step_idx"})
)
integer_columns = ["scale", "step_idx"]
df_combined[integer_columns] = df_combined[integer_columns].astype(int)

In [191]:
n_scales = df_combined["scale"].unique().size
n_simulations = df_combined["simulation_idx"].max() + 1
n_steps = df_combined["step_idx"].max() + 1

In [192]:
def to_image(df: pd.DataFrame, scale: int):
    window_width = 1000 * scale
    positions = df[["posx_m", "posy_m"]].dropna().to_numpy() - window_width / 2
    weights = df["exist_prob"].fillna(1.0).to_numpy()
    return position_image(
        window_width=1000 * scale,
        size=128 * scale,
        sigma=10.0,
        target_positions=positions,
        device="cuda",
        weights=weights,
    )

results = []
groupby_columns = ["simulation_idx", "step_idx", "scale"]
gb = df_combined.query("scale == 1").groupby(groupby_columns, as_index=False, sort=True)
for (idx, df) in tqdm(gb, total=n_scales * n_simulations * n_steps):
    simulation_idx, step_idx, scale = idx
    truth = df[df["truth"] == "truth"]
    filter = df[df["truth"] == "filter"]

    truth_positions = truth[["posx_m", "posy_m"]].dropna().to_numpy()
    filter_positions = filter[["posx_m", "posy_m"]].dropna().to_numpy()
    ospa = compute_ospa(truth_positions, filter_positions, 500, 2)

    truth_image = to_image(truth, scale)
    filter_image = to_image(filter, scale)
    mse = (truth_image - filter_image).pow(2).mean().item()

    results.append(dict(
        scale=scale,
        simulation_idx=simulation_idx,
        step_idx=step_idx,
        ospa=ospa,
        mse=mse,
        filter=filter_name,
        cardinality_truth=len(truth_positions),
        cardinality_estimate=filter["exist_prob"].sum()
    ))
df_results = pd.DataFrame(results)

 20%|██        | 10000/50000 [00:46<03:06, 213.91it/s]


In [193]:
df_results.to_csv(f"../data/out/{filter_name}_summary.csv", index=False)